# **CIFAR10**

In [1]:
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as  F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
print("Torch version: ", torch. __version__)

####################################################################
# Set Device
####################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)


####################################################################
# Dataset Class
####################################################################
class CIFAR10_dataset(Dataset):

    def __init__(self, partition = "train"):

        print("\nLoading CIFAR10 ", partition, " Dataset...")
        self.partition = partition
        if self.partition == "train":
            self.data = torchvision.datasets.CIFAR10('.data/', 
                                                     train=True,
                                                     download=True)
        else:
            self.data = torchvision.datasets.CIFAR10('.data/', 
                                                     train=False,
                                                     download=True)
        print("\tTotal Len.: ", len(self.data), "\n", 50*"-")

        self.transform_train = torchvision.transforms.Compose([
            torchvision.transforms.RandomCrop(32, padding=4),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                mean=(0.4914, 0.4822, 0.4465),
                std=(0.2023, 0.1994, 0.2010)
            ),
            torchvision.transforms.RandomErasing(p=0.5)
        ])
        self.transform_test = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                mean=(0.4914, 0.4822, 0.4465),
                std=(0.2023, 0.1994, 0.2010)
            ),
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        # Image
        image = self.data[idx][0]
        # Apply transforms
        if self.partition == "train":
            image_tensor = self.transform_train(image)
        else:
            image_tensor = self.transform_test(image)

        # Label
        label = torch.tensor(self.data[idx][1])
        # label = F.one_hot(label, num_classes=10).float()

        return {"img": image_tensor, "label": label}

train_dataset = CIFAR10_dataset(partition="train")
test_dataset = CIFAR10_dataset(partition="test")

####################################################################
# DataLoader Class
####################################################################

batch_size = 100
num_workers = 0 # multiprocessing.cpu_count()-1
print("Num workers", num_workers)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=num_workers)

####################################################################
# Neural Network Class
####################################################################
class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super().__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(
            in_channels, growth_rate,
            kernel_size=3, padding=1, bias=False
        )

    def forward(self, x):
        out = self.conv(self.relu(self.bn(x)))
        return torch.cat([x, out], dim=1)
class DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate):
        super().__init__()
        layers = []
        channels = in_channels
        for _ in range(num_layers):
            layers.append(DenseLayer(channels, growth_rate))
            channels += growth_rate
        self.block = nn.Sequential(*layers)
        self.out_channels = channels

    def forward(self, x):
        return self.block(x)
class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(
            in_channels, out_channels,
            kernel_size=1, bias=False
        )
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        out = self.conv(self.relu(self.bn(x)))
        out = self.pool(out)
        return out
class DenseNet100(nn.Module):
    def __init__(self, num_classes=10, growth_rate=12):
        super().__init__()

        num_channels = 2 * growth_rate  # 24

        self.conv1 = nn.Conv2d(
            3, num_channels, kernel_size=3,
            padding=1, bias=False
        )

        self.block1 = DenseBlock(16, num_channels, growth_rate)
        num_channels = self.block1.out_channels
        self.trans1 = TransitionLayer(num_channels, num_channels // 2)
        num_channels //= 2

        self.block2 = DenseBlock(16, num_channels, growth_rate)
        num_channels = self.block2.out_channels
        self.trans2 = TransitionLayer(num_channels, num_channels // 2)
        num_channels //= 2

        self.block3 = DenseBlock(16, num_channels, growth_rate)
        num_channels = self.block3.out_channels

        self.bn = nn.BatchNorm2d(num_channels)
        self.fc = nn.Linear(num_channels, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.trans1(self.block1(x))
        x = self.trans2(self.block2(x))
        x = self.block3(x)
        x = F.relu(self.bn(x))
        x = F.adaptive_avg_pool2d(x, 1)
        x = torch.flatten(x, 1)
        return self.fc(x)


# Instantiating the network and printing its architecture
num_classes = 10
net = DenseNet100(num_classes)
print(net)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Params: ", count_parameters(net))

####################################################################
# Training settings
####################################################################

# Training hyperparameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=5e-4, momentum=0.9)
epochs = 100

# El Scheduler reducirá el LR siguiendo una curva de coseno
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

####################################################################
# Training
####################################################################

# Load model in GPU
net.to(device)

print("\n---- Start Training ----")
best_accuracy = -1
best_epoch = 0
for epoch in range(epochs):


    # TRAIN NETWORK
    train_loss, train_correct = 0, 0
    net.train()
    with tqdm(iter(train_dataloader), desc="Epoch " + str(epoch), unit="batch") as tepoch:
        for batch in tepoch:
            
            # Returned values of Dataset Class
            images = batch["img"].to(device)
            labels = batch["label"].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = net(images)
            loss = criterion(outputs, labels)

            # Calculate gradients
            loss.backward()

            # Update gradients
            optimizer.step()

            # one hot -> labels
            pred = torch.argmax(outputs, dim=1)
            train_correct += pred.eq(labels).sum().item()

            # print statistics
            train_loss += loss.item()
        scheduler.step()

    train_loss /= (len(train_dataloader.dataset) / batch_size)

    # TEST NETWORK
    test_loss, test_correct = 0, 0
    net.eval()
    with torch.no_grad():
      with tqdm(iter(test_dataloader), desc="Test " + str(epoch), unit="batch") as tepoch:
          for batch in tepoch:

            images = batch["img"].to(device)
            labels = batch["label"].to(device)

            # Forward
            outputs = net(images)
            test_loss += criterion(outputs, labels)

            # one hot -> labels
            pred = torch.argmax(outputs, dim=1)

            test_correct += pred.eq(labels).sum().item()

    test_loss /= (len(test_dataloader.dataset) / batch_size)
    test_accuracy = 100. * test_correct / len(test_dataloader.dataset)

    print("[Epoch {}] Train Loss: {:.6f} - Test Loss: {:.6f} - Train Accuracy: {:.2f}% - Test Accuracy: {:.2f}%".format(
        epoch + 1, train_loss, test_loss, 100. * train_correct / len(train_dataloader.dataset), test_accuracy
    ))

    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_epoch = epoch

        # Save best weights
        torch.save(net.state_dict(), "best_model.pt")

print("\nBEST TEST ACCURACY: ", best_accuracy, " in epoch ", best_epoch)

Torch version:  2.10.0.dev20251124+cu130
Device:  cuda

Loading CIFAR10  train  Dataset...
	Total Len.:  50000 
 --------------------------------------------------

Loading CIFAR10  test  Dataset...
	Total Len.:  10000 
 --------------------------------------------------
Num workers 0
DenseNet100(
  (conv1): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (block1): DenseBlock(
    (block): Sequential(
      (0): DenseLayer(
        (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv): Conv2d(24, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (1): DenseLayer(
        (bn): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv): Conv2d(36, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (2): DenseLayer(
        (bn): BatchNorm2d(48, eps

Test 0: 100%|██████████| 100/100 [00:06<00:00, 15.21batch/s]


[Epoch 1] Train Loss: 1.526815 - Test Loss: 1.326108 - Train Accuracy: 43.66% - Test Accuracy: 51.79%


Test 1: 100%|██████████| 100/100 [00:06<00:00, 15.20batch/s]


[Epoch 2] Train Loss: 1.096562 - Test Loss: 1.623620 - Train Accuracy: 60.77% - Test Accuracy: 52.09%


...

[Epoch 98] Train Loss: 0.070404 - Test Loss: 0.226036 - Train Accuracy: 97.78% - Test Accuracy: 93.68%


Test 98: 100%|██████████| 100/100 [00:07<00:00, 13.94batch/s]


[Epoch 99] Train Loss: 0.073122 - Test Loss: 0.226856 - Train Accuracy: 97.72% - Test Accuracy: 93.77%


Test 99: 100%|██████████| 100/100 [00:07<00:00, 14.22batch/s]

[Epoch 100] Train Loss: 0.072649 - Test Loss: 0.228423 - Train Accuracy: 97.65% - Test Accuracy: 93.79%

BEST TEST ACCURACY:  93.87  in epoch  90
